# Answering Business Questions Using SQL
***
## Introduction

This analysis uses the [Chinook](https://github.com/lerocha/chinook-database) database. The Chinook database is a fictional digital media shop based on actual data from an iTunes library. We will be looking at sales in terms of individual tracks vs. albums, along with sales by country and many more variables. The goal is to see what Chinook can do as a company to maximize profits for its digital media shop.

## Data Overview

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook-unmodified.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook-unmodified.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


The database has 11 tables that range from the album, artist, customer, and genre. There are also tables for information on employees working for Chinook, customers, and how many are purchasing a specific track or album, along with invoices to keep track of purchases. There's a plethora of information that helps give us a good idea about the scope and parameters of the database for analyzing all angles to see if and how the business is thriving.

## Top Genres

In [10]:
%%sql

WITH
usa_invoice_line AS
    (
     SELECT
        il.invoice_line_id invoice_line_id,
        il.track_id track_id,
        c.country country
     FROM invoice_line il
     INNER JOIN invoice inv ON inv.invoice_id = il.invoice_id
     INNER JOIN customer c on c.customer_id = inv.customer_id
     WHERE c.country = "USA"   
    )
SELECT
    g.name genre,
    COUNT(uil.invoice_line_id) count,
    ROUND(CAST(COUNT(uil.invoice_line_id)*100 AS Float)/(SELECT COUNT(invoice_line_id) FROM usa_invoice_line), 1) percent
FROM usa_invoice_line uil
INNER JOIN track t ON t.track_id = uil.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY genre
ORDER BY count DESC
LIMIT 15;

 * sqlite:///chinook-unmodified.db
Done.


genre,count,percent
Rock,561,53.4
Alternative & Punk,130,12.4
Metal,124,11.8
R&B/Soul,53,5.0
Blues,36,3.4
Alternative,35,3.3
Pop,22,2.1
Latin,22,2.1
Hip Hop/Rap,20,1.9
Jazz,14,1.3


Here are the top 15 genres in descending order, with **Rock** at the top.

| Artist Name          | Genre   |
| -------------------- | ------- |
| Regal                | Hip-Hop |
| Red Tone             | Punk    |
| Meteor and the Girls | Pop     |
| Slim Jim Bites       | Blues   |

After seeing the top 15 genres, we're tasked with taking these four albums and picking the top three albums we should purchase for Chinook's virtual store. The chart above shows that the store should choose **Red Tone - Punk** since Alternative & Punk is the second most popular genre. Then they should select **Slim Jim Bites - Blues**, then **Meteor and the Girls - Pop**.

## Analyzing Employee Sales Performance

In [15]:
%%sql

WITH customer_support_rep_sales AS
    (
    SELECT
        i.customer_id,
        c.support_rep_id,
        SUM(i.total) AS total
    FROM invoice AS i
    INNER JOIN customer AS c on c.customer_id = i.customer_id
    GROUP BY 1,2
    )

SELECT
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    ROUND(SUM(csrs.total)) AS total_sales
FROM customer_support_rep_sales AS csrs
INNER JOIN employee AS e on e.employee_id = csrs.support_rep_id
GROUP BY 1;

 * sqlite:///chinook-unmodified.db
Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1732.0
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1394.0


We can see the top three employees here, with Jane Peacock with the highest total sales. We have Margaret Park and Steve Johnson not too far behind. One thing to note, though, in terms of top employees is that Jane was hired at an earlier date than both of them. Because of this, it allows Jane a higher chance of getting more sales compared to Margaret and Steve. Even accounting for the hiring date, Jane is slightly better when it comes to sales than Margaret, but Steve is on par to pass Jane since he was hired six and a half months after her and is on pace to pass her when he reaches the same amount of employment time as Jane.

## Analyzing Sales by Country

In [58]:
%%sql

WITH sales_per_customer AS   (SELECT c.country, 
                                     c.customer_id,
                                     ROUND(SUM(i.total), 0) total,
                                     COUNT(i.invoice_id) num_sales
                                FROM customer c
                                     INNER JOIN invoice i 
                                     ON i.customer_id=c.customer_id
                               GROUP BY 1, 2),

     all_countries AS        (SELECT c.country,
                                     COUNT(c.customer_id) customers,
                                     ROUND(SUM(spc.total), 0) total_sales,
                                     ROUND(AVG(spc.num_sales), 1) sales_per_customer,
                                     ROUND(AVG(spc.total), 0) average_order
                                FROM customer c
                                     INNER JOIN sales_per_customer spc 
                                     ON spc.customer_id = c.customer_id
                               GROUP BY 1),  
    
     only_1_customer AS      (SELECT COUNT(country) country,
                                     COUNT(customers) customers,
                                     SUM(total_sales) total_sales,
                                     ROUND(AVG(sales_per_customer), 0) sales_per_customer, 
                                     ROUND(AVG(average_order), 0) average_order
                                FROM all_countries
                               WHERE customers=1),  
        
    other_countries AS       (SELECT CASE 
                                     WHEN country = (SELECT country 
                                                       FROM only_1_customer)       
                                                       THEN "Other"
                                     END AS country, 
                                     customers,
                                     total_sales, 
                                     sales_per_customer, 
                                     average_order
                                FROM only_1_customer),
    
     more_than_1_customer AS (SELECT * 
                                FROM all_countries 
                               WHERE customers > 1
                               ORDER BY 1),
        
     united_unsorted AS      (SELECT * FROM other_countries
                               UNION 
                              SELECT * FROM more_than_1_customer)   
                              
SELECT country,
       customers,
       ROUND(customers * 100.0/(SELECT SUM(customers) 
                                      FROM united_unsorted), 0) pct_customers,
       total_sales,
       ROUND(total_sales * 100.0/(SELECT SUM(total_sales) 
                                      FROM united_unsorted), 0) pct_sales,       
       sales_per_customer,
       average_order
  FROM (SELECT united_unsorted.*,
               CASE
               WHEN united_unsorted.country = "Other" THEN 1
               ELSE 0
               END AS sort
          FROM united_unsorted)
 ORDER BY sort ASC, 4 DESC;


 * sqlite:///chinook-unmodified.db
Done.


country,customers,pct_customers,total_sales,pct_sales,sales_per_customer,average_order
USA,13,22.0,1038.0,22.0,10.1,80.0
Canada,8,14.0,535.0,11.0,9.5,67.0
Brazil,5,8.0,427.0,9.0,12.2,85.0
France,5,8.0,388.0,8.0,10.0,78.0
Germany,4,7.0,334.0,7.0,10.3,84.0
Czech Republic,2,3.0,274.0,6.0,15.0,137.0
United Kingdom,3,5.0,245.0,5.0,9.3,82.0
Portugal,2,3.0,185.0,4.0,14.5,93.0
India,2,3.0,183.0,4.0,10.5,92.0
Other,15,25.0,1093.0,23.0,10.0,73.0


Here we're analyzing the number of customers each country has, total sales, the percentage of sales they have as a whole when grouped with other countries, sales per customer, and the average order value. Since there are some countries in our database with only one customer, it's better to group them into their own category. Hence, the **Other** name under the country column is all those countries with only one customer. It makes more sense to put the country's name in this list if they have at least two or more customers.

We can see that the USA is far ahead of anyone regarding customers and total sales, making up 22% of customers and sales for the whole data. One piece of data that stands out here is that the Czech Republic has the highest average order value by far and away. Adding onto that, what's interesting is that they only have two customers, which means those two customers were hungry for new music to listen to. Other countries such as Portugal and India had the second and third highest average order amounts while only having two customers each. With this in mind, these three countries are perfect for targeting those countries' music audiences to maximize profit since they're so willing to purchase a lot of songs/albums.

## Total Tracks Sold

In [59]:
%%sql

WITH all_vs_sold AS (SELECT COUNT(DISTINCT(t.track_id)) all_tracks, 
                            COUNT(DISTINCT(il.track_id)) sold_tracks
                       FROM track t
                            LEFT JOIN invoice_line il 
                            ON il.track_id = t.track_id)

SELECT all_vs_sold.*,
       ROUND(sold_tracks * 100.0 / all_tracks, 0) pct_sold
  FROM all_vs_sold;

 * sqlite:///chinook-unmodified.db
Done.


all_tracks,sold_tracks,pct_sold
3503,1806,52.0


Here we have the total amount of tracks that Chinook holds, which is 3,503. The number of total tracks sold was 1,806, which is 52% of the tracks, which is pretty decent. The thing with this data is that customers can only either purchase individual tracks or whole albums. Since Chinook has to be specific with what tracks and albums they want to get the rights to sell in their store, the issue is that customers aren't allowed to purchase an album along with individual tracks all in the same purchase. Management is considering letting customers purchase only the most popular tracks from each album from record companies instead of being able to purchase every track from an album. With this in mind, we need to see customers' buying habits in terms of whether they're buying more tracks than albums or vice versa. 

## Individual Tracks vs Albums

In [60]:
%%sql

WITH invoice_album AS   (SELECT il.invoice_id,                                     
                                t.album_id
                           FROM invoice_line il
                                LEFT JOIN track t 
                                ON t.track_id = il.track_id
                          GROUP BY 1),

album_vs_tracks AS (SELECT ia.*,
                                CASE
                                WHEN (SELECT il.track_id 
                                        FROM invoice_line il
                                       WHERE il.invoice_id = ia.invoice_id
                                      EXCEPT                                          
                                      SELECT t.track_id 
                                        FROM track t
                                       WHERE t.album_id = ia.album_id) IS NULL
                             
                                 AND (SELECT t.track_id 
                                        FROM track t
                                       WHERE t.album_id = ia.album_id                                         
                                      EXCEPT                                          
                                      SELECT il.track_id 
                                        FROM invoice_line il
                                       WHERE il.invoice_id = ia.invoice_id) IS NULL
                                   
                                THEN "Albums"
                                ELSE "Tracks"
                                END AS purchase                             
                           FROM invoice_album ia)
                           
SELECT purchase,
       COUNT(*) num_invoices,
       ROUND(COUNT(*) * 100.0 /(SELECT COUNT(*)
                                FROM invoice), 0) pct
FROM album_vs_tracks
GROUP BY 1
ORDER BY 2 DESC;

 * sqlite:///chinook-unmodified.db
Done.


purchase,num_invoices,pct
Tracks,500,81.0
Albums,114,19.0


As we can see, individual tracks sell just over four times as much as albums. With this data, it's clear that Chinook should focus on selling the most popular individual tracks and less on selling albums. We can see also from the chart near the top what the best-selling genres are. Rock, Alternative & Punk, and Metal are the top three genres, so Chinook could focus on selling the top tracks from those genres. We can also look at the most popular artists to get a better idea of how to go about selling individual tracks.

## Most Popular Artists

In [61]:
%%sql

SELECT ar.name artist_name,
       COUNT(DISTINCT(pt.playlist_id)) num_playlists
  FROM artist ar
       INNER JOIN album al 
       ON al.artist_id = ar.artist_id   
        
       INNER JOIN track t 
       ON t.album_id = al.album_id
    
       INNER JOIN playlist_track pt 
       ON pt.track_id = t.track_id    
 GROUP BY 1
 ORDER BY 2 DESC
 LIMIT 5;

 * sqlite:///chinook-unmodified.db
Done.


artist_name,num_playlists
Eugene Ormandy,7
The King's Singers,6
English Concert & Trevor Pinnock,6
Berliner Philharmoniker & Herbert Von Karajan,6
Academy of St. Martin in the Fields & Sir Neville Marriner,6


Here we have the top five artists regarding how many playlists they're in. The more playlists an artist is in, the more popular they are. We can see here that **Eugene Ormandy** is in the most playlists. Close behind are other artists such as **The King's Singers** and **English Concert & Trevor Pinnock**. What we can do is see what genre these artists make music in.

In [62]:
%%sql

SELECT DISTINCT(g.name) genre
  FROM genre g
       INNER JOIN track t 
       ON t.genre_id = g.genre_id   
    
       INNER JOIN album al 
       ON al.album_id = t.album_id
    
       INNER JOIN artist ar 
       ON ar.artist_id = al.artist_id    
 WHERE ar.name = "Eugene Ormandy" 
    OR ar.name = "The King's Singers"
    OR ar.name = "English Concert & Trevor Pinnock"
    OR ar.name = "Berliner Philharmoniker & Herbert Von Karajan" 
    OR ar.name = "Academy of St. Martin in the Fields & Sir Neville Marriner";

 * sqlite:///chinook-unmodified.db
Done.


genre
Classical


Interestingly these artists are comprised in the **Classical** genre. Interestingly, if we look at the chart at the top, the classical genre was the 14th most popular genre. So, seeing the most popular artists in the Chinook database being from the 14th most popular genre isn't what we expected. With this information, it's not a good idea to focus on these top artists and selling their individual tracks since, yes, they're in the highest number of playlists, but classical music is nowhere near one of the top genres.

## Conclusion

After analyzing the Chinook database and trying to find answers to help the company maximize profits, we've come up with the following findings:

- They should purchase the three albums mentioned earlier, which are **Red Tone**, which was in the Punk genre, **Slim Jim Bites** in the Blues genre, and **Meteor and the Girls**, which was in the Pop genre. These three genres were the highest among the four in the overall most popular genres in Chinook's digital media shop. They need to capitalize on what's popular and push these three albums.

- Of the three sales agents, **Jane Peacock** was the most successful, with **Steve Johnson** on pace with her. **Margaret Park** was the least successful of the three sales agents, so one idea would be to focus on Jane and Steve taking on the more essential customers regarding how much money those customers are spending. If the customer is looking to buy a lot of individual tracks which leads to them spending more money than the average customer, statistically speaking, it would be wiser to have Jane or Steve deal with those more higher profile customers that're willing to spending more money since Chinook needs those purchases to go through.

- Out of all the countries, the USA has the highest percentage of total customers and sales, so focusing on the USA when it comes to bringing more attention to helping those customers out with their purchases will help in an increase in revenue. Another aspect to consider when it comes to countries and music is that the Czech republic had the highest average order value by quite a lot. Portugal and India did as well, but those countries only had two customers who bought from Chinook. What could be done with these countries since there's clear potential for higher sales since the customers are willing to spend a lot is to implement ads more in those countries.

- Customers, by far and away, buy individual tracks compared to albums, so focusing on purchasing the most popular individual tracks for their digital media shop is imperative to maximize profits. This is because customers have to weigh whether they want to purchase an album or individual tracks more since that's their policy that you can't double dip. So, purchasing popular individual tracks from the record companies from the most popular genres will help them maximize profits. We did see that the artists in the most playlists were in the classical genre, but the classical genre is the 14th most popular and is near the bottom of the list. Because of this, it's noted to not focus on those artists and push their individual tracks as a great way to maximize profits.

- About half of the individual tracks weren't purchased, which could come down to them being in less popular genres. Yes, it's important to have a range of tracks across multiple genres, but if we have to be picky in the number of individual tracks we can purchase from record companies, we want to focus on what's popular genre-wise. We can see from the classical genre, which is a perfect example, that the most popular artists, when it came to the number of playlists, were a part of a genre that most people didn't care for.